### this notebook saves the metrics (static ones; and lss ones) for the missing v1.5 db.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.5_-0.1cuts/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.5_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

#### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.65_nodither_nside256.csv
eg_footprint_stats_y3_limi25.25_nodither_nside256.csv
eg_footprint_stats_y6_limi25.619999999999997_nodither_nside256.csv
eg_footprint_stats_y10_limi25.9_nodither_nside256.csv
{'yr1_limi24.65': 'Y1 (i$>$24.65) ', 'yr3_limi25.25': 'Y3 (i$>$25.25) ', 'yr6_limi25.619999999999997': 'Y6 (i$>$25.619999999999997) ', 'yr10_limi25.9': 'Y10 (i$>$25.9) '}
dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.65
3 25.25
6 25.619999999999997
10 25.9


#### get the order of the dbs

In [5]:
#data['yr10_limi25.9'].dbname.values

In [6]:
# need to do this since input order is provided for only some of the dbs
df = pd.DataFrame(['baseline_samefilt_v1.5_10yrs']
                  , columns=['name'])
df.to_csv('%s/given_missing_v1.5.csv' % outdir, index=False)

In [7]:
df_in_order = pd.read_csv('%s/given_missing_v1.5.csv' % outdir, )
df_in_order

,name
0,baseline_samefilt_v1.5_10yrs


In [8]:
# check some things:
len( list( df_in_order['name'] ) ), len( np.unique( df_in_order['name'] ) )

(1, 1)

In [9]:
yr_key = [f for f in data.keys() if f.__contains__('yr1_')][0]
yr_key

'yr1_limi24.65'

#### find the indices to get the order

In [10]:
# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( data[yr_key]['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[85]


In [11]:
# check we have the right indices
print( data[yr_key]['dbname'].values[28],  df_in_order['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

bulges_bs_v1.5_10yrs baseline_samefilt_v1.5_10yrs
1 1


#### save the data in the order in which to save things

In [12]:
data.keys()

dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])

In [13]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             df_in_order[colname] = data[yr_lim_tag][colname].values[ind]

In [14]:
df_in_order

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,baseline_samefilt_v1.5_10yrs,15303.37,25.11,0.18,15336.41,25.72,0.14,15336.26,26.07,0.13,15321.36,26.35,0.13


In [15]:
cols = df_in_order.columns.tolist()
cols

['name',
 'Y1 effective survey area ',
 'median Y1 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'Y3 effective survey area ',
 'median Y3 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'Y6 effective survey area ',
 'median Y6 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'Y10 effective survey area ',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area']

#### realized that there's too much finagling when sharing data so its best to reorder columns to make sure the order of the columns is the same.

In [16]:
new_cols = ['name',
            'median Y1 $i$-band coadded depth in effective survey area',
            'median Y3 $i$-band coadded depth in effective survey area',
            'median Y6 $i$-band coadded depth in effective survey area',
            'median Y10 $i$-band coadded depth in effective survey area',
            'std in Y1 $i$-band coadded depth in effective survey area',
            'std in Y3 $i$-band coadded depth in effective survey area',
            'std in Y6 $i$-band coadded depth in effective survey area',
            'std in Y10 $i$-band coadded depth in effective survey area',
            'Y1 effective survey area ',
            'Y3 effective survey area ',
            'Y6 effective survey area ',
            'Y10 effective survey area ',]
new_cols

['name',
 'median Y1 $i$-band coadded depth in effective survey area',
 'median Y3 $i$-band coadded depth in effective survey area',
 'median Y6 $i$-band coadded depth in effective survey area',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area',
 'Y1 effective survey area ',
 'Y3 effective survey area ',
 'Y6 effective survey area ',
 'Y10 effective survey area ']

In [17]:
df_in_order = df_in_order[new_cols]
df_in_order

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,baseline_samefilt_v1.5_10yrs,25.11,25.72,26.07,26.35,0.18,0.14,0.13,0.13,15303.37,15336.41,15336.26,15321.36


In [18]:
# save the metrics
fname = 'static_metrics_missing.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom and ngal

In [19]:
# clean up the give_list dataframe
for key in df_in_order.keys():
    if key.__contains__('Y'):
        df_in_order = df_in_order.drop(key, axis=1)
        
df_in_order

,name
0,baseline_samefilt_v1.5_10yrs


#### read in the lss fom values

In [20]:
data_dir_base = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.5_-0.1cuts/lss_fom/'

In [21]:
data_dir = '%s/os_bias_overplots/' % data_dir_base
[f for f in os.listdir(data_dir) if f.__contains__('fomdata') and f.__contains__('2021-03-08')] 

['2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_86cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv',
 '2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_86cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv']

In [22]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y1_') and f.__contains__('2021-03-08')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y1 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_86cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv
[65]


In [23]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y10_') and f.__contains__('2021-03-08')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y10 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_86cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv
[65]


In [24]:
df_in_order


,name,Y1 lss fom,Y10 lss fom
0,baseline_samefilt_v1.5_10yrs,0.910804,0.979782


#### now read in the ngal values

In [25]:
ngal_data_dir = data_dir_base

ngal_data = {}
for yr in [1, 10]:
    ngal_data[yr] =  np.zeros(len(df_in_order.name))
    folders = [f for f in os.listdir( ngal_data_dir ) if f.__contains__('_Y%s_' % yr) and f.__contains__('i<') ]
    for i, folder in enumerate( folders ):
        out = pd.read_csv('%s/%s/ngal-final_total/ngal_NoDither.txt' % (ngal_data_dir, folder), header=None, delimiter=' ') 
        dbname, ngal = out[0].values[0], out[1].values[0]
        ind = np.where( df_in_order['name'].values == '%s' % dbname)[0]
        ngal_data[yr][ind] = ngal
    print('read in %s folders for Y%s' % (i+1, yr) )

read in 86 folders for Y1
read in 86 folders for Y10


In [26]:
for yr in ngal_data:
    df_in_order['Y%s Ngal (0.66<z<1.0)' % yr] = ngal_data[yr]

In [27]:
df_in_order

,name,Y1 lss fom,Y10 lss fom,Y1 Ngal (0.66<z<1.0),Y10 Ngal (0.66<z<1.0)
0,baseline_samefilt_v1.5_10yrs,0.910804,0.979782,447000000.0,960000000.0


In [28]:
fname = 'lss_metrics_missing.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)